In [ ]:
import os
import gc
import joblib
import random
import numpy as np
import pandas as pd
from pathlib import Path
from argparse import Namespace
from scipy.stats import pearsonr
from collections import defaultdict

import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler

import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor

In [ ]:
# [0.0625, 0.0625, 0.125, 0.25, 0.5] for 5 fold
def weighted_average(a):
    w = [0.1, 0.1, 0.1, 0.3, 0.4]
#     n = len(a)
#     for j in range(1, n + 1):
#         j = 2 if j == 1 else j
#         w.append(1 / (2**(n + 1 - j)))
    return np.average(a, weights = w, axis=0)

In [ ]:
features = [f'f_{i}' for i in range(300)]

# # mirror
features.remove('f_112')
# # too high VIF
# features.remove('f_61')
# features.remove('f_164')
# ori_features = features.copy()
# len(features)

In [ ]:
# select_features = ['f_74', 'f_153', 'f_108', 'f_145', 'f_63', 
#                    'f_22', 'f_225', 'f_183', 'f_62', 'f_241',
#                    'f_142', 'f_8', 'f_231', 'f_165', 'f_170']

# select_index = [74, 153, 108, 145, 63, 
#                 22, 225, 183, 62, 241, 
#                 142, 8, 231, 165, 170]
# pred_features = []
# last_features = []
# for feature in select_features:
#     pred_features.append('pred_'+feature)
#     last_features.append('last_'+feature)
# features.extend(pred_features)

In [ ]:
import ubiquant
env = ubiquant.make_env()
iter_test = env.iter_test()

folds = 5

scaler = joblib.load('../input/sixmodels/scaler.joblib')
models = [joblib.load(f'../input/sixmodels/lgbm_seed2022_{fold}.pkl') for fold in range(folds)]

for (test, sample_prediction_df) in iter_test:
    x = scaler.fit_transform(test[features].values)
#     last_x = np.copy(x[:, select_index])
#     pred_x = np.copy(x[:, select_index])
#     last_x = np.concatenate(([last_x[0]], last_x[:-1]), axis=0)
#     pred_x = np.concatenate((pred_x[1:], [pred_x[-1]]), axis=0)
#     X_test = np.concatenate((x, last_x, pred_x), axis=1)
    X_test = x
    final_pred = [models[fold].predict(X_test) for fold in range(folds)]
    sample_prediction_df['target'] = weighted_average((np.stack(final_pred)))
    env.predict(sample_prediction_df)
    display(sample_prediction_df)